In [1]:
from torch_geometric.datasets import Planetoid
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv,ChebConv, SAGEConv, GATConv, GINConv
from torch.nn import Linear

In [2]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0] 

In [3]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, activationGNN, activationOut):
        super(GCN, self).__init__()
        torch.manual_seed(42)
        
        # Initialize the layers
        self.conv1 = GCNConv(dataset.num_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.fullyConnected = Linear(hidden_channels, dataset.num_classes)
        self.activationGNN = activationGNN
        self.activationOut = activationOut

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.activationGNN(x)
        x = F.dropout(x, p=0.5, training=self.training)

        x = self.conv2(x, edge_index)
        x = self.activationGNN(x)
        x = F.dropout(x, p=0.5, training=self.training)

        if self.activationOut == torch.softmax:
            x = self.activationOut(self.fullyConnected(x), dim=1)
        else:
            x = self.activationOut(self.fullyConnected(x))
        return x

In [4]:
import torch.nn.utils.prune as prune

Weights are randomly initialized when creating the model

In [5]:
model = GCN(hidden_channels=64, activationGNN=F.leaky_relu, activationOut=torch.softmax)
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.CrossEntropyLoss()

In [6]:
# Access the weights of each layer
conv1_weights = model.conv1.lin.weight.clone()
conv2_weights = model.conv2.lin.weight.clone()

In [7]:
conv2_weights[0][0]

tensor(0.0517, grad_fn=<SelectBackward0>)

In [8]:
def train():
      model.train()
      optimizer.zero_grad() 
      out = model(data.x, data.edge_index)  
      # Używamy tylko nodów z labelkami do obliczenia funkcji straty
      loss = criterion(out[data.train_mask], data.y[data.train_mask])  
      loss.backward() 
      optimizer.step()
      return loss

In [9]:
def test():
      model.eval()
      out = model(data.x, data.edge_index)
      pred = out.argmax(dim=1)  
      test_correct = pred[data.test_mask] == data.y[data.test_mask]  
      test_acc = int(test_correct.sum()) / int(data.test_mask.sum())  
      return test_acc, pred

In [10]:
# Funckja do trenowania modelu przez zadaną liczbę epok
def final_train(epochs):
    losses = []
    for epoch in range(0, epochs):
        loss = train()
        losses.append(loss)
    return losses

In [11]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 0.00%


In [12]:
loss = final_train(1000)

In [13]:
test()

(0.785, tensor([3, 4, 4,  ..., 5, 3, 3]))

In [ ]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [ ]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [30]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 20.00%


In [31]:
loss = final_train(1000)
test()

(0.779, tensor([3, 4, 4,  ..., 5, 3, 3]))

In [32]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [33]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.4,  # Prune 20% of the parameters
)

In [34]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 52.00%


In [35]:
loss = final_train(1000)
test()

(0.777, tensor([3, 4, 4,  ..., 5, 3, 3]))

In [36]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [37]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.6,  # Prune 20% of the parameters
)

In [38]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 80.80%


In [39]:
loss = final_train(1000)
test()

(0.791, tensor([3, 4, 4,  ..., 1, 3, 3]))

In [40]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [41]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.8,  # Prune 20% of the parameters
)

In [42]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 92.32%


In [43]:
loss = final_train(1000)
test()

(0.78, tensor([3, 4, 4,  ..., 1, 3, 3]))

<font size="5">Winning tickety </font>

In [14]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [15]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.2,  # Prune 20% of the parameters
)

In [16]:
print(
    "Global sparsity: {:.2f}%".format(
        100. * float(
            torch.sum(model.conv1.lin.weight == 0)
            + torch.sum(model.conv2.lin.weight == 0)
        )
        / float(
            model.conv1.lin.weight.nelement()
            + model.conv2.lin.weight.nelement()
        )
    )
)

Global sparsity: 20.00%


In [ ]:
def recoverOriginalWeightsW1():
    clone = model.conv1.lin.weight.clone()
    for i, neuron in enumerate(model.conv1.lin.weight):
        for j, elem in enumerate(neuron) :
#             print(clone[i][j])
#             print(conv1_weights[i][j])
            if elem != 0: # pomijamy sprunowane wagi
                clone[i][j].fill_(conv1_weights[i][j].detach())
    return clone                    

In [ ]:
def recoverOriginalWeightsW2():
    clone = model.conv2.lin.weight.clone()
    for i, neuron in enumerate(model.conv2.lin.weight):
        for j, elem in enumerate(neuron) :
            if elem != 0: # pomijamy sprunowane wagi
                clone[i][j].fill_(conv2_weights[i][j].detach())
    return clone                    

In [ ]:
def printW(tensor):
    counter = 0
    for i, neuron in enumerate(tensor):
        for j, elem in enumerate(neuron) :
            if counter == 10:
                break
            print(tensor[i][j])
            print(conv1_weights[i][j])
            print(model.conv1.lin.weight[i][j])
            counter += 1

In [ ]:
model.conv1.lin.weight = recoverOriginalWeightsW1()

<font size="5">Z jakiegoś powodu tutaj nie mogę zrobic pruningu na dwoch warstwach bo wywala kernel, musze sprawdzic osobno. Edit: osobno udało mi się tylko raz sprunowac pierwsza warstwe, ale to chyba bez sensu wrzucac do sprawka. Próbowałem też losowej reinicjalizacji wag, też wywala.</font>

In [17]:
def randomReinitalizeW1(original_tensor):
    clone = original_tensor.clone()
    for i, neuron in enumerate(original_tensor):
        for j, elem in enumerate(neuron) :
            if elem != 0: # pomijamy sprunowane wagi
                tensor = torch.empty(1)
                tensor.uniform_(-0.1, 0.1)
                clone[i][j].fill_(tensor.item())
    return clone                    

In [18]:
model.conv1.lin.weight = randomReinitalizeW1(model.conv1.lin.weight)

In [ ]:
loss = final_train(1000)

In [ ]:
test()

In [22]:
parameters_to_prune = (
    (model.conv1.lin, 'weight'),  # Prune the weights of layer1
    (model.conv2.lin, 'weight'),  # Prune the weights of layer2
)

In [23]:
prune.global_unstructured(
    parameters_to_prune,
    pruning_method=prune.L1Unstructured,  # Use L1-norm based pruning
    amount=0.4,  # Prune 20% of the parameters
)

In [24]:
model.conv1.lin.weight = recoverOriginalWeightsW1()

In [ ]:
loss = final_train(1000)

In [ ]:
test()

<font size="5">Dalsza część została wykonana w collabie ponieważ tam udało się zrobić pruning.</font>